In [42]:
import numpy as np
import math

In [43]:
S0 = 100       #current stock price
K = 100        #strike
T = 0.75       #expiry time
r = 0.03       #interest rate
sigma = 0.35   #volatility
opttype = 0    #0 for a call, otherwise a put
Nsteps = 500   #number of timesteps

In [44]:
#payoff at t=T for European call/put option
 
def European(S0,K,T,r,sigma,opttype,Nsteps):
    delt = T/Nsteps
    
    #Tree parameters
    u = np.exp(sigma * np.sqrt(delt))
    d = 1./u
    a = np.exp( r*delt)
    p = (a-d)/(u-d)
    
    #payoff price
    W = S0 * d**(np.arange(Nsteps,-1,-1)) * u**(np.arange(Nsteps+1))
    if (opttype == 0):  #call
        W = np.maximum( W - K, 0);
    else: #put
        W = np.maximum( K - W, 0);
    for i in np.arange(Nsteps, 0, -1):
        W = np.exp(-r*delt)*(p*W[1:i+1] + (1-p)*W[0:i])
    return(W[0])
        

In [45]:
#payoff at t=T for American call option
 
def cAmerican(S0,K,T,r,sigma,Nsteps):
    delt = T/Nsteps
    
    #Tree parameters
    u = np.exp(sigma * np.sqrt(delt))
    d = 1./u
    a = np.exp( r*delt)
    p = (a-d)/(u-d)
    
    #payoff price
    W = S0 * d**(np.arange(Nsteps,-1,-1)) * u**(np.arange(Nsteps+1))
    W = np.maximum( W - K, 0) #call
    current = W
    for i in np.arange(Nsteps, 0, -1):
        W = np.exp(-r*delt)*(p*W[1:i+1] + (1-p)*W[0:i])
        current = current[0:i]*u
        W = np.maximum(W, np.maximum( current - K, 0))
    return(W[0])

In [46]:
#payoff at t=T for American put option
 
def pAmerican(S0,K,T,r,sigma,Nsteps):
    delt = T/Nsteps
    
    #Tree parameters
    u = np.exp(sigma * np.sqrt(delt))
    d = 1./u
    a = np.exp( r*delt)
    p = (a-d)/(u-d)
    
    #payoff price
    W = S0 * d**(np.arange(Nsteps,-1,-1)) * u**(np.arange(Nsteps+1))
    current = W
    W = np.maximum( K - W, 0) #put

    for i in np.arange(Nsteps, 0, -1):
        W = np.exp(-r*delt)*(p*W[1:i+1] + (1-p)*W[0:i])
        current = current[0:i]*u
        W = np.maximum(W, np.maximum( K - current, 0))
    return(W[0])

In [47]:
pAmerican(100,100,0.75,0.03,0.35,500)

11.026708354516346

In [48]:
European(100,100,0.75,0.03,0.35,0,500)

13.051177442418886

In [49]:
European(100,100,0.75,0.03,0.35,1,500)

10.82630116174909

In [50]:
cAmerican(100,100,0.75,0.03,0.35,500)

13.051177442418886

In [51]:
pAmerican(100,100,0.75,0.03,0.35,500)

11.026708354516346

In [52]:
from scipy.stats import norm
import numpy as np

In [53]:
#adapted from Octave's financial toolkit
def blsprice(Price, Strike, Rate, Time, Volatility):
    sigma_sqrtT = Volatility * np.sqrt (Time)

    d1 = 1 / sigma_sqrtT * (np.log(Price / Strike) + (Rate + Volatility**2 / 2) * Time)
    d2 = d1 - sigma_sqrtT

    phi1 = norm.cdf(d1)
    phi2 = norm.cdf(d2)
    disc = np.exp (-Rate * Time)
    F    = Price * np.exp ((Rate) * Time)

    Call = disc * (F * phi1 - Strike * phi2)
    Put  = disc * (Strike * (1 - phi2) + F * (phi1 - 1))
    return Call, Put

In [54]:
#adapted from Octave's financial toolkit
def blsdelta(Price, Strike, Rate, Time, Volatility):
    d1 = 1 / (Volatility * np.sqrt(Time)) * (np.log (Price / Strike) + (Rate + Volatility**2 / 2) * Time)

    phi = norm.cdf(d1)

    CallDelta = phi
    PutDelta = phi - 1
    return CallDelta, PutDelta

In [55]:
blsprice (100, 100, 0.03, 0.75, 0.35)

(13.057147530853024, 10.832271250186647)

In [56]:
blsdelta(100, 100, 0.03, 0.75, 0.35)

(0.5893157536060705, -0.41068424639392953)

In [57]:
#table for European call options

import tabulate
delt1 = T/500
delt2 = T/1000
delt3 = T/2000
delt4 = T/4000
delt5 = T/8000

value1 = European(100,100,0.75,0.03,0.35,0,500)
value2 = European(100,100,0.75,0.03,0.35,0,1000)
value3 = European(100,100,0.75,0.03,0.35,0,2000)
value4 = European(100,100,0.75,0.03,0.35,0,4000)
value5 = European(100,100,0.75,0.03,0.35,0,8000)

change2_1 = value2 - value1
change3_2 = value3 - value2
change4_3 = value4 - value3
change5_4 = value5 - value4

ratio3 = change2_1 / change3_2
ratio4 = change3_2 / change4_3
ratio5 = change4_3 / change5_4

exact_v = blsprice (100, 100, 0.03, 0.75, 0.35)[0]

ctable = [[delt1,value1],
         [delt2,value2,change2_1],
         [delt3,value3,change3_2,ratio3],
         [delt4,value4,change4_3,ratio4],
         [delt5,value5,change5_4,ratio5,exact_v]]

col = ["Delta_t", "Value", "Change", "Ratio", "blsprice"]
print(tabulate.tabulate(ctable, headers=col))

  Delta_t    Value       Change    Ratio    blsprice
---------  -------  -----------  -------  ----------
0.0015     13.0512
0.00075    13.0542  0.00298467
0.000375   13.0557  0.00149262   1.99962
0.0001875  13.0564  0.000746378  1.99981
9.375e-05  13.0568  0.000373207  1.99991     13.0571


In [58]:
#table for European put options

import tabulate
pdelt1 = T/500
pdelt2 = T/1000
pdelt3 = T/2000
pdelt4 = T/4000
pdelt5 = T/8000

pvalue1 = European(100,100,0.75,0.03,0.35,1,500)
pvalue2 = European(100,100,0.75,0.03,0.35,1,1000)
pvalue3 = European(100,100,0.75,0.03,0.35,1,2000)
pvalue4 = European(100,100,0.75,0.03,0.35,1,4000)
pvalue5 = European(100,100,0.75,0.03,0.35,1,8000)

pchange2_1 = pvalue2 - pvalue1
pchange3_2 = pvalue3 - pvalue2
pchange4_3 = pvalue4 - pvalue3
pchange5_4 = pvalue5 - pvalue4

pratio3 = pchange2_1 / pchange3_2
pratio4 = pchange3_2 / pchange4_3
pratio5 = pchange4_3 / pchange5_4

pexact_v = blsprice (100, 100, 0.03, 0.75, 0.35)[1]

ptable = [[pdelt1,pvalue1],
         [pdelt2,pvalue2,pchange2_1],
         [pdelt3,pvalue3,pchange3_2,pratio3],
         [pdelt4,pvalue4,pchange4_3,pratio4],
         [pdelt5,pvalue5,pchange5_4,pratio5,pexact_v]]

pcol = ["Delta_t", "Value", "Change", "Ratio", "blsprice"]
print(tabulate.tabulate(ptable, headers=pcol))

  Delta_t    Value       Change    Ratio    blsprice
---------  -------  -----------  -------  ----------
0.0015     10.8263
0.00075    10.8293  0.00298467
0.000375   10.8308  0.00149262   1.99962
0.0001875  10.8315  0.000746378  1.99981
9.375e-05  10.8319  0.000373207  1.99991     10.8323


In [59]:
#table for American put options

import tabulate
pAdelt1 = T/500
pAdelt2 = T/1000
pAdelt3 = T/2000
pAdelt4 = T/4000
pAdelt5 = T/8000

pAvalue1 = pAmerican(100,100,0.75,0.03,0.35,500)
pAvalue2 = pAmerican(100,100,0.75,0.03,0.35,1000)
pAvalue3 = pAmerican(100,100,0.75,0.03,0.35,2000)
pAvalue4 = pAmerican(100,100,0.75,0.03,0.35,4000)
pAvalue5 = pAmerican(100,100,0.75,0.03,0.35,8000)

pAchange2_1 = pAvalue2 - pAvalue1
pAchange3_2 = pAvalue3 - pAvalue2
pAchange4_3 = pAvalue4 - pAvalue3
pAchange5_4 = pAvalue5 - pAvalue4

pAratio3 = pAchange2_1 / pAchange3_2
pAratio4 = pAchange3_2 / pAchange4_3
pAratio5 = pAchange4_3 / pAchange5_4

pAtable = [[pAdelt1,pAvalue1],
          [pAdelt2,pAvalue2,pAchange2_1],
          [pAdelt3,pAvalue3,pAchange3_2,pAratio3],
          [pAdelt4,pAvalue4,pAchange4_3,pAratio4],
          [pAdelt5,pAvalue5,pAchange5_4,pAratio5]]

pAcol = ["Delta_t", "Value", "Change", "Ratio"]
print(tabulate.tabulate(pAtable, headers=pAcol))

  Delta_t    Value       Change    Ratio
---------  -------  -----------  -------
0.0015     11.0267
0.00075    11.0286  0.00188109
0.000375   11.0295  0.000941684  1.99758
0.0001875  11.03    0.000468239  2.01112
9.375e-05  11.0302  0.000233725  2.00337


The order of accuracy from the table is 1. This agrees with the theory above. The calculation process is provided in the next page. 